## Project 2 Wrangling OpenStreetMap Data (Chicago)

Data from MapZen

In [243]:
#Libraries
import xml.etree.ElementTree as ET
from collections import defaultdict
import pprint
import re
import codecs
import json
import pymongo as mongo
import pandas as pd
import numpy as np
import sys

In [244]:
#OSM file
OSMFILE = "chicago_city.osm"

## What are we dealing with?

Lets get a list of what information we'd like to grab from the XML data source

In [245]:
tagList = []

# Loop through the OSM file and store fields to a list
def research_audit(osmfile):
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                addToTagList(tag)
    osm_file.close()

# print list as panda series with count of occurence of each field
def printTagList():
    np.set_printoptions(threshold=sys.maxsize)
    pdList = pd.Series(tagList)
    with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
        pprint.pprint(pdList.value_counts())
    
research_audit(OSMFILE)    
printTagList()

building                                 174484
chicago:building_id                      129830
addr:street                              111769
addr:housenumber                         111287
addr:street:name                         109964
addr:street:type                         109597
addr:street:prefix                       109387
building:levels                           90288
highway                                   24112
name                                      18225
service                                    7520
operator                                   5979
tiger:cfcc                                 5082
tiger:county                               5071
source                                     5068
railway                                    5014
tiger:name_base                            4878
tiger:name_type                            4823
tiger:name_direction_prefix                4738
tiger:zip_left                             4384
amenity                                 

## What are these?

The above list has over 400 keys and all of them might not be useful.
However, I've chosen some keys along with an example value that I find should be audited for 
consistency, accuracy, and uniformity.

| Key           | Example Values |
| ------------- |:-------------:|
| addr:street   | South Kilpatrick Avenue, West Chicago Ave |
| phone  | 312-226-0670, (312) 226-1988 |


The listed fields above present some issues. For "addr:street" we have inconsistent street types such as Avenue and Ave. These will need to be formatted into a single street type. Also, "phone" should be formated to a single style.


## Dealing with street names

In [84]:
#list to check against for inconsistant street types. These are the desired street types, non-abbreviated
expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Access"]

#Regex to find street types gets last word in a string
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

def check_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)


def is_street_type(elem):
    return ((elem.attrib['k'] == "addr:street"))

# loop through and populate a dictionary containing non-expected street types
def audit_research_street_type(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_type(tag):
                    check_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print((k, v) )

In [80]:
st_types = audit_research_street_type(OSMFILE)
print_sorted_dict(st_types)    


('1425', {'N Lake Shore Dr #1425'})
('1850', {'E Madison St #1850'})
('2105', {'North Michigan Avenue # 2105'})
('400', {'South Michigan Avenue # 400'})
('500', {'Delaware Pl #500'})
('510', {'W Madison St #510'})
('575', {'N LaSalle St, #575'})
('Access', {'North Breakwater Access'})
('Ave', {'West Chicago Ave'})
('Blvd', {'W Jackson Blvd', 'West Jackson Blvd'})
('Bouevard', {'West Garfield Bouevard'})
('Clinton', {'South Clinton'})
('Dr', {'North Upper Wacker Dr'})
('LaSalle', {'North LaSalle'})
('level)', {'North State Street (Lower level)'})
('Market', {'West Fulton Market'})
('NW', {'Linnean Ave. NW'})
('Park', {'East Groveland Park', 'East Woodland Park'})
('Plaza', {'North Riverside Plaza', 'W Merchandise Mart Plaza', 'Merchandise Mart Plaza', 'East Carver Plaza', 'South Riverside Plaza', 'West Wolf Point Plaza'})
('South', {'East Riverwalk South'})
('St', {'Adams St', 'W Lake St', 'N LaSalle St', 'W 63rd St', 'W 18th St', 'S Dearborn St', 'N Clark St', 'E Oak St'})
('St.', {'W.

We can bring some consistency to street names by converting all abbraviations to the full word even if only one instance of it used:

* Blvd, Bouevard(typo found) -> Boulevard
* Ave -> Avenue
* Dr -> Drive
* St, St. -> Street


In [107]:
found_street_changes = defaultdict(set)
     
# Map to convert abbreviated street type to its respective full street type
mapping = { "St": "Street",
            "St.": "Street",
            "Ave": "Avenue",
            "Ave.": "Avenue",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Blvd" : "Boulevard",
            "Bouevard" : "Boulevard",
            "Pl" : "Place"
            }
# similar to last time but this time replace the street type and track this change in a dictionary
def check_street_type(street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected and street_type in mapping.keys():
            replaced = street_type_re.sub(mapping[street_type], street_name)
            street_name = replaced
            found_street_changes[street_type].add(street_name)
    return street_name

def audit_street_type(osmfile):
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_type(tag):
                    check_street_type(tag.attrib['v'])
    osm_file.close()
    


In [108]:
audit_street_type(OSMFILE)
print_sorted_dict(found_street_changes)

('Ave', {'West Chicago Avenue'})
('Blvd', {'West Jackson Boulevard', 'W Jackson Boulevard'})
('Bouevard', {'West Garfield Boulevard'})
('Dr', {'North Upper Wacker Drive'})
('St', {'W Lake Street', 'E Oak Street', 'N Clark Street', 'N LaSalle Street', 'S Dearborn Street', 'W 18th Street', 'Adams Street', 'W 63rd Street'})
('St.', {'W. Lake Street', 'W. Madison Street'})


## Dealing with phone numbers

Using the same technique we're going to replace all different phone number strings to only include the digits with no spaces. For example: 1234567891

In [132]:
# regex to find different formated phone numbers
phone_format_reg = re.compile(r'\+?1?\s*\(?-*\.*(\d{3})\)?\.*-*\s*(\d{3})\.*-*\s*(\d{4})$')     
found_phone_changes = defaultdict(set)

# repalce all formated phone numbers with just numerical versions only
def format_phone_num(phone):
    m = phone_format_reg.search(phone)
    if m:
        numbers_list = re.findall('\d+', phone)
        if numbers_list[0] is "1":
            numbers_list.pop(0)
        elif numbers_list[0].startswith('1',0,1):
            numbers_list[0] = numbers_list[0][:0] + numbers_list[0][(0+1):]
        numbers = "".join(numbers_list)
        found_phone_changes[m.group()].add(numbers)
        phone = numbers
    return phone

def is_phone_num(elem):
    return ((elem.attrib['k'] == "phone"))

def audit_phone(osmfile):
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_num(tag):
                    format_phone_num(tag.attrib['v'])
    osm_file.close()
    

In [270]:
audit_phone(OSMFILE)
print_sorted_dict(found_phone_changes)

(' 312-780-1314', {'3127801314'})
('(312) 222-9899', {'3122229899'})
('(312) 226-1988', {'3122261988'})
('(312) 226-4017', {'3122264017'})
('(312) 238-9650', {'3122389650'})
('(312) 239-3603', {'3122393603'})
('(312) 251-7009', {'3122517009'})
('(312) 267-0571', {'3122670571'})
('(312) 322-6777', {'3123226777'})
('(312) 329-1000', {'3123291000'})
('(312) 332-2244', {'3123322244'})
('(312) 337.3903', {'3123373903'})
('(312) 369-7900', {'3123697900'})
('(312) 421-3631', {'3124213631'})
('(312) 431-8029', {'3124318029'})
('(312) 487-1648', {'3124871648'})
('(312) 489-5097', {'3124895097'})
('(312) 561-5500', {'3125615500'})
('(312) 629-0099', {'3126290099'})
('(312) 644-2090', {'3126442090'})
('(312) 742-2410', {'3127422410'})
('(312) 746-5092', {'3127465092'})
('(312) 746-5560', {'3127465560'})
('(312) 747-3481', {'3127473481'})
('(312) 747-6707', {'3127476707'})
('(312) 747-7676', {'3127477676'})
('(312) 932-9600', {'3129329600'})
('(312) 943-2220', {'3129432220'})
('(312) 945-8510', {'

## Putting it all together and exporting to JSON


In [210]:
lower_colon = re.compile(r'^(\b\S+\.?)*:(\b\S+\.?)*$', re.IGNORECASE)
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

CREATED = [ "version", "changeset", "timestamp", "user", "uid"]

def handle_address(elem, key, node):
    value = elem.attrib['v']
    if is_street_type(elem):
        value = check_street_type(value)
    add_dict("address", key, value, node)

def add_dict(dict_name, key, value, node):
    if dict_name in node and type(node[dict_name]) is dict:
        node[dict_name][key] = value
    else:
        node[dict_name] = {}
        node[dict_name][key] = value
    return node

def handle_tag(elem, node):
    if problemchars.search(elem.attrib['k']):
        return node
    elif (elem.attrib['k'] == "phone"):
        node["phone"] = format_phone_num(elem.attrib['v'])
    elif (lower_colon.search(elem.attrib['k'])):
        colon_split = elem.attrib['k'].split(':')
        if colon_split[0] == 'addr':
            handle_address(elem, colon_split[1], node)
        else:
            add_dict(colon_split[0], colon_split[1], elem.attrib['v'], node)
    else:
        node[elem.attrib['k']] = elem.attrib['v']
    return node

def handle_pos(value, node):
    if ("pos" in node) :
        node["pos"].append(float(value))
    else:
        node["pos"] = []
        node["pos"].append(float(value))

def handle_elem(elem, node):
    node["type"] = elem.tag
    for attr, value in elem.attrib.items():
        if ( attr == "lat" or attr == "lon" ):
            handle_pos(value, node)
        elif attr in CREATED:
            add_dict("created", attr, value, node)
        else:
            node[attr] = value

def shape_element(element):
    node = {}
    if element.tag == "node" or element.tag == "way" :
        handle_elem(element, node)
        for tag in element.iter("tag"):
            handle_tag(tag, node)
        return node
    else:
        return None


def process_map(file_in, pretty = False):
    file_out = "mongo_chicago.json".format(file_in)
    data = []
    with codecs.open(file_out, "w") as fo:
        for _, element in ET.iterparse(file_in):
            el = shape_element(element)
            if el:
                data.append(el)
                if pretty:
                    fo.write(json.dumps(el, indent=2)+"\n")
                else:
                    fo.write(json.dumps(el) + "\n")
    return data

In [271]:
process_map(OSMFILE, True)
print("Finished creating JSON file")

finished creating JSON file


## Data Overview

In [212]:
client = mongo.MongoClient("mongodb://localhost:27017")
client.database_names()
db = client.local
chi = db["mongo_chicago"]
chi.find_one()

{'_id': ObjectId('59f92970d5022e753d78504d'),
 'created': {'changeset': '8323836',
  'timestamp': '2011-06-02T20:24:34Z',
  'uid': '451048',
  'user': 'bbmiller',
  'version': '9'},
 'id': '20216902',
 'pos': [41.8895268, -87.6393324],
 'source': 'PGS',
 'type': 'node'}

### Size of files


| Name        | Size           |
| ------------- |:----------:|
| chicago_city.osm| 312 MB |
| mongo_chicago.json| 353 MB |


In [213]:
## Info on the DB
print(db.command("dbstats"))

{'db': 'local', 'collections': 2, 'views': 0, 'objects': 1441966, 'avgObjSize': 228.703004093023, 'dataSize': 329781956.0, 'storageSize': 91361280.0, 'numExtents': 0, 'indexes': 2, 'indexSize': 14282752.0, 'ok': 1.0}


### Distinct users

In [214]:
len(chi.find().distinct("created.user"))

650

### Number of ways

In [216]:
chi.find({"type":"way"}).count()

206527

### Number of nodes

In [217]:
chi.find({"type":"node"}).count()

1235428

### Number of cafes

In [218]:
chi.find({"amenity":"cafe"}).count()

138

### Top 10 Cuisines

In [231]:
list(chi.aggregate([{"$match": {
            "cuisine": { "$exists": True}
        }},{"$group" : {"_id" : "$cuisine", "count" : {"$sum" : 1}}},
    {"$sort" : {"count" : -1}}, {"$limit":10}]))

[{'_id': 'burger', 'count': 78},
 {'_id': 'sandwich', 'count': 43},
 {'_id': 'coffee_shop', 'count': 36},
 {'_id': 'mexican', 'count': 29},
 {'_id': 'pizza', 'count': 26},
 {'_id': 'chicken', 'count': 17},
 {'_id': 'american', 'count': 16},
 {'_id': 'italian', 'count': 15},
 {'_id': 'chinese', 'count': 13},
 {'_id': 'japanese', 'count': 8}]

### Top Amenities

In [233]:
list(chi.aggregate([{"$match": {
            "amenity": { "$exists": True}
        }},{"$group" : {"_id" : "$amenity", "count" : {"$sum" : 1}}},
    {"$sort" : {"count" : -1}}]))

[{'_id': 'parking', 'count': 1440},
 {'_id': 'place_of_worship', 'count': 611},
 {'_id': 'school', 'count': 411},
 {'_id': 'restaurant', 'count': 357},
 {'_id': 'fast_food', 'count': 206},
 {'_id': 'bicycle_rental', 'count': 178},
 {'_id': 'cafe', 'count': 138},
 {'_id': 'fuel', 'count': 124},
 {'_id': 'fountain', 'count': 86},
 {'_id': 'bank', 'count': 74},
 {'_id': 'bar', 'count': 73},
 {'_id': 'pharmacy', 'count': 60},
 {'_id': 'pub', 'count': 49},
 {'_id': 'drinking_water', 'count': 37},
 {'_id': 'library', 'count': 37},
 {'_id': 'bench', 'count': 34},
 {'_id': 'toilets', 'count': 32},
 {'_id': 'fire_station', 'count': 32},
 {'_id': 'bicycle_parking', 'count': 28},
 {'_id': 'atm', 'count': 25},
 {'_id': 'theatre', 'count': 25},
 {'_id': 'post_office', 'count': 22},
 {'_id': 'waste_basket', 'count': 16},
 {'_id': 'hospital', 'count': 15},
 {'_id': 'university', 'count': 15},
 {'_id': 'embassy', 'count': 15},
 {'_id': 'police', 'count': 14},
 {'_id': 'college', 'count': 14},
 {'_id':

## Areas for Improvement
### The Name Field:

An area to improve in this analysis is the handling of the names field.
Note that there is variablity in terms of what users input in that field. The expected and most common is actually the name of a particular location like a park's name or store name. However sometimes we see that this field is used for intersections and address as well.

In [286]:
#regex to find anything contiaining &. & because it is used to many intersection names
contains_ampersand = re.compile(r'[&]')
amp_list = list(chi.find({'name':contains_ampersand}).distinct("name"))
# don't want print the entire list since it's huge.
pprint.pprint(amp_list[:10])


["Ada's Famous Deli & Restaurant",
 "Pizano's Pizza & Pasta",
 'Michigan & Madison-Monroe',
 'State & Madison',
 'Midtown Kitchen & Bar',
 'Solidarity Drive & Adler Planetarium',
 'State & Monroe',
 'J&M Tap',
 'Lake Shore Dr & Ohio St',
 'Bear & Bull']


In [240]:
## Note not all of these are intersections but a it looks like a majority of them are
print(len(amp_list))

2363


In [287]:
## regex to find contianing the word street but not the character &
contains_street = re.compile(r'^(?=.*street)(?!.*&).*',re.IGNORECASE)

st_list = list(chi.find({'name':contains_street}).distinct("name"))
pprint.pprint(st_list[:10])

['Streeterville',
 '21st Street',
 '16th Street',
 '18th Street',
 'Ash Street Mp 5.6',
 'CP 54th Street',
 'CP 59th Street',
 'CP Lumber Street',
 'Museum Campus / 11th Street',
 'Van Buren Street']


In [242]:
## Note this is just for the word street, consider other common types like avenue and lane
print(len(st_list))

424


We could attempt something similar to what we did for addr:street field before. For intersections I think they could be better served as their own field, like the highway field. For street names they should be merged with the street names field. However, it wont be that simple since as mentioned before the name field is highly variable. Above I only matched for names containing an ampersand and then for names that contain the word street and not an ampersand. This is not very precise at all since there are names that are not intersections that contain ampersands like "H&R Block". So to do this would not be an easy feat using the methods above.


### Cross Validation:

Another area for improving this analysis would be to add cross validation for these inputs from other data sources. The data provided from MapZen is user driven like wikipedia however unlike wikipedia there isn't an easy way to check the input's claim from sources listed.

Along with this we can also cross validate certain inputs with another data source. A popular option for chicago is the city's data portal (https://data.cityofchicago.org/). So for example, the nodes detailing anything from the Chicago Transit Authority (CTA) can be validated against the data on this site. 

In [288]:
contains_transit = re.compile(r'^(?=.*Chicago Transit Authority)(?!.*&).*',re.IGNORECASE)

transit = list(chi.find({"operator":contains_transit}))
pprint.pprint(transit[:5])

[{'_id': ObjectId('59f92970d5022e753d785304'),
  'alt_name': 'Chicago/Franklin',
  'created': {'changeset': '51870343',
              'timestamp': '2017-09-09T06:45:26Z',
              'uid': '1237170',
              'user': 'Zol87',
              'version': '10'},
  'id': '102715649',
  'name': 'Chicago',
  'operator': 'Chicago Transit Authority',
  'platforms': '2',
  'pos': [41.8962625, -87.6357112],
  'railway': 'station',
  'type': 'node'},
 {'_id': ObjectId('59f92970d5022e753d785608'),
  'created': {'changeset': '51871051',
              'timestamp': '2017-09-09T07:25:27Z',
              'uid': '1237170',
              'user': 'Zol87',
              'version': '18'},
  'id': '258026049',
  'name': 'UIC-Halsted',
  'operator': 'Chicago Transit Authority',
  'platforms': '1',
  'pos': [41.875474, -87.6496755],
  'railway': 'station',
  'station': 'subway',
  'type': 'node',
  'wheelchair': 'no'},
 {'_id': ObjectId('59f92970d5022e753d785609'),
  'address': {'housenumber': '430', 'st

CTA dataset link:
https://data.cityofchicago.org/Transportation/CTA-List-of-CTA-Datasets/pnau-cf66

The value of this would be wonderful for adding confidence in the dataset. However, implementing this would be a challenge. One possible way is that we can query the second database when building the JSON file. The problem with that though is that would drastically increase the time to build the JSON file. Not only that but building out the query to the validation dataset would not be simple either, since there is a lot variation in certain fields like name. So we would have to be careful in building out the query.